In [1]:
# !pip install -r requirements.txt
import os
import sys

%cd /app
# sys.path.append('..')

/app


#### - Setup MLflow

In [2]:
from utils import mlflow_utils
import mlflow

# uri=mlflow_utils.setup_mlflow()
uri = "http://localhost:5000"
mlflow.set_tracking_uri(uri)

## 1- ML pipeline (sklearn)

### 1.1- sklearn

In [3]:
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor

os.environ["GIT_PYTHON_REFRESH"] = "quiet"

#### 1.1.1- data preparation

In [4]:
# Initialize data
train_data = [[1, 4, 5, 6], [4, 5, 6, 7], [30, 40, 50, 60]]
train_labels = [10, 20, 30]
eval_data = [[2, 4, 6, 8], [1, 4, 50, 60]]

#### 1.1.2- model training

In [5]:
# Initialize CatBoostRegressor
params = {
    "iterations": 2,
    "learning_rate": 1,
    "depth": 2,
    "allow_writing_files": False,
}
model = Pipeline([("reg", CatBoostRegressor(**params))])

print(f" -> model training ...")
# Fit model
model.fit(train_data, train_labels)

 -> model training ...
0:	learn: 6.1237244	total: 47.4ms	remaining: 47.4ms
1:	learn: 4.5927933	total: 47.7ms	remaining: 0us


Pipeline(steps=[('reg',
                 <catboost.core.CatBoostRegressor object at 0x7f079363ddc0>)])

In [6]:
# Log parameters and fitted model
print(f"  -> Log parameters and fitted model")
with mlflow.start_run() as run:
    mlflow.log_params(params)
    mlflow.sklearn.log_model(model, artifact_path="model")
    model_uri = mlflow.get_artifact_uri("model")

  -> Log parameters and fitted model


MlflowException: API request to http://localhost:5000/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f07910282e0>: Failed to establish a new connection: [Errno 111] Connection refused'))

#### 1.1.3- model inference

In [ ]:
print(f" -> model inference ...")
# Load model
loaded_model = mlflow.sklearn.load_model(model_uri)

In [ ]:
# Get predictions
preds = loaded_model.predict(eval_data)
print("predictions:", preds)